In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_batch = 60000
test_batch = 10000

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
print(device)

cuda


In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,)),
                               torchvision.transforms.Lambda(lambda x: x.view(-1,1))
                             ])),
  batch_size=train_batch, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,)),
                               torchvision.transforms.Lambda(lambda x: x.view(-1,1))
                             ])),
  batch_size=test_batch, shuffle=False)

# **LMU, BMU, LLMU**

In [ ]:
!pip install nengolib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from nengolib.signal import Identity,cont2discrete
from nengolib.synapses import LegendreDelay
import numpy as np
from scipy.special import comb


class LMU(nn.Module):
    def __init__(self, input_size, hidden_size, memory_size, theta, matrix_type='pl',discretizer = 'zoh',nonlinearity='sigmoid', A_learnable = False, B_learnable = False):
        super(LMU, self).__init__()

        ### SIZE
        self.k = input_size
        self.n = hidden_size
        self.d = memory_size

        ### PARAMETERS
        self.Wx = nn.Parameter(torch.Tensor(self.n,self.k))
        self.Wh = nn.Parameter(torch.Tensor(self.n,self.n))
        self.Wm = nn.Parameter(torch.Tensor(self.n,self.d))
        self.ex = nn.Parameter(torch.Tensor(1,self.k))
        self.eh = nn.Parameter(torch.Tensor(1,self.n))
        self.em = nn.Parameter(torch.Tensor(1,self.d))

        ### A,B MATRIX ----- FIX??

        if matrix_type=='pl':   #For Legendre Memory Unit
            order=self.d
            Q = np.arange(order, dtype=np.float64)
            R = (2 * Q + 1)[:, None] / theta
            j, i = np.meshgrid(Q, Q)
            A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
            B = (-1.0) ** Q[:, None] * R
            C = np.ones((1, order))
            D = np.zeros((1,))
            self._ss = cont2discrete((A, B, C, D), dt=0.01, method=discretizer)
            self._A = self._ss.A
            self._B = self._ss.B
        elif matrix_type=='p':  #For Pade Memory Unit
            order=self.d
            Q=np.arange(order,dtype=np.float64)
            V=(order+Q+1)*(order-Q)/(Q+1)/theta
            A=np.zeros([order,order],dtype=np.float64)
            B=np.zeros([order,1],dtype=np.float64)
            A[0,:]=-V[0]
            A[1:order,0:order-1]=np.diag(V[1:order])
            B[0]=V[0]
            C = np.ones((1, order))
            D = np.zeros((1,))
            self._ss = cont2discrete((A, B, C, D), dt=0.01, method=discretizer)
            self._A = self._ss.A
            self._B = self._ss.B
        elif matrix_type=='pb':  #For Bernstein Memory Unit
            order=self.d
            Q = np.arange(order, dtype=np.float64)
            R = (2 * Q + 1)[:, None] / theta
            j, i = np.meshgrid(Q, Q)
            A_leg = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
            B_leg = (-1.0) ** Q[:, None] * R
            C = np.ones((1, order))
            D = np.zeros((1,))
            M=np.zeros([order,order],dtype=np.float64)
            M_inv=np.zeros([order,order],dtype=np.float64)
            n=order-1 #degree of polynomial
            for j in range(0,n+1):
              for k in range(0,n+1):
                ll=max(0,j+k-n)
                ul=min(j,k)+1
                sum=0.0
                for i in range(ll,ul):
                  sum=sum+((-1.0)**(k+i))*(comb(k,i)**2)*comb(n-k,j-i)
                M[j,k]=sum/comb(n,j)

                sum=0.0
                for i in range(0,j+1):
                  sum=sum+(-1.0)**(j+i)*comb(j,i)**2/comb(n+j,k+i)
                M_inv[j,k]=(2*j+1)/(n+j+1)*comb(n,k)*sum

            M=10*np.tanh(M/10)
            M_inv=10*np.tanh(M_inv/10)

            A_1=np.matmul(M,A_leg)
            A=np.matmul(A_1,M_inv)
            B=np.matmul(M,B_leg)

            self._ss = cont2discrete((A, B, C, D), dt=0.01, method=discretizer)
            self._A = self._ss.A
            self._B = self._ss.B

        ### NON-LINEARITY
        self.nl = nonlinearity
        if self.nl == 'sigmoid':
            self.act = nn.Sigmoid()
        elif self.nl == 'tanh':
            self.act = nn.Tanh()
        else:
            self.act = nn.ReLU()

        ### INITIALIZATION
        torch.nn.init.xavier_normal_(self.Wm)    ##### FIGURE THIS OUT!!
        torch.nn.init.xavier_normal_(self.Wx)
        torch.nn.init.xavier_normal_(self.Wh)
        torch.nn.init.zeros_(self.em)
        torch.nn.init.uniform_(self.ex, -np.sqrt(3/self.d), np.sqrt(3/self.d))
        torch.nn.init.uniform_(self.eh, -np.sqrt(3/self.d), np.sqrt(3/self.d))


        #### TRIAL
        self.register_buffer('AT', torch.Tensor(self._A))
        self.register_buffer('BT', torch.Tensor(self._B))
        if A_learnable:
            self.AT = nn.Parameter(self.AT)
        if B_learnable:
            self.BT = nn.Parameter(self.BT)


    def forward(self,x,hm):

        h,m = hm
        u = F.linear(x,self.ex)+F.linear(h,self.eh)+F.linear(m,self.em)
        new_m = F.linear(m,self.AT) + F.linear(u,self.BT)
        new_h = self.act(F.linear(x,self.Wx)+F.linear(h,self.Wh)+F.linear(new_m,self.Wm))

        return new_h,new_m


class ASSVMU(nn.Module):

    def __init__(self, input_size, hidden_size, memory_size, theta, name='garbage', discretizer = 'zoh',nonlinearity='sigmoid', 
                        A_learnable = False, B_learnable = False, activate=False):
        super(ASSVMU, self).__init__()
        
        ### SIZE
        self.k = input_size
        self.n = hidden_size
        self.d = memory_size
        

        ### PARAMETERS
        self.Wx = nn.Parameter(torch.Tensor(self.n,self.k))
        self.Wh = nn.Parameter(torch.Tensor(self.n,self.n))
        self.Wm = nn.Parameter(torch.Tensor(self.n,self.d))
        self.ex = nn.Parameter(torch.Tensor(1,self.k))
        self.eh = nn.Parameter(torch.Tensor(1,self.n))
        self.em = nn.Parameter(torch.Tensor(1,self.d))

        ### A,B MATRIX ----- FIX??
        order=self.d
        Q = np.arange(order, dtype=np.float64)
        R = (2 * Q + 1)[:, None] / theta
        j, i = np.meshgrid(Q, Q)
        A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
        B = (-1.0) ** Q[:, None] * R
        C = np.ones((1, order))
        D = np.zeros((1,))
        self._ss = cont2discrete((A, B, C, D), dt=0.01, method=discretizer)
        self._A = self._ss.A
        self._B = self._ss.B

        ### NON-LINEARITY
        self.nl = nonlinearity
        if self.nl == 'sigmoid':
            self.act = nn.Sigmoid()
        elif self.nl == 'tanh':
            self.act = nn.Tanh()
        else:
            self.act = nn.ReLU()

        ### NN
        self.fc = nn.Linear(self.n,self.n)

        if activate:
            self.nn_act = self.act
        else:
            self.nn_act = nn.LeakyReLU(1.0) #Identity Function

        ### INITIALIZATION
        torch.nn.init.xavier_normal_(self.Wm)    ##### FIGURE THIS OUT!!
        torch.nn.init.xavier_normal_(self.Wx)
        torch.nn.init.xavier_normal_(self.Wh)
        torch.nn.init.zeros_(self.em)
        torch.nn.init.uniform_(self.ex, -np.sqrt(3/self.d), np.sqrt(3/self.d))
        torch.nn.init.uniform_(self.eh, -np.sqrt(3/self.d), np.sqrt(3/self.d))


        #### TRIAL
        self.register_buffer('AT', torch.Tensor(self._A))
        self.register_buffer('BT', torch.Tensor(self._B))
        if A_learnable:
            self.AT = nn.Parameter(self.AT)
        if B_learnable:
            self.BT = nn.Parameter(self.BT)

    def forward(self,x,hm):

        h,m = hm 
        u = F.linear(x,self.ex)+F.linear(h,self.eh)+F.linear(m,self.em)
        new_m = F.linear(m,self.AT) + F.linear(u,self.BT)
        new_h = self.act(F.linear(x,self.Wx)+F.linear(h,self.Wh)+F.linear(new_m,self.Wm))
        new_h = self.nn_act(self.fc(new_h))
        return new_h,new_m

In [ ]:
lmu = LMU(input_size=1,hidden_size=256,memory_size=50,theta=784,matrix_type='pb').to(device)
linear = nn.Linear(256,10).to(device)

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lmu.parameters())
optimizer_linear = optim.Adam(linear.parameters())


In [ ]:
def train_model(epoch):

	lmu.train()
	linear.train()
	running_loss_train = 0.0
	batch_size = 1000

	train_accuracy = []

	for batches in range(int(train_batch/batch_size)):
		

		inputs = images_train[batches*batch_size : (batches+1)*batch_size]
		labels = classes_train[batches*batch_size : (batches+1)*batch_size]
		
		hx = torch.zeros(batch_size, 256).to(device) # (batch, hidden_size)
		cx = torch.zeros(batch_size, 50).to(device)

		inputs = inputs.permute(1,0,2)
		optimizer.zero_grad()
		optimizer_linear.zero_grad()

		for i in range(inputs.size()[0]):
			hx,cx = lmu(inputs[i],(hx,cx))

		output = linear(hx)

		out = F.log_softmax(output, dim=1)

		loss =  F.cross_entropy(out,labels)


		loss.backward()
		optimizer.step()
		optimizer_linear.step()

		running_loss_train += loss.item() * inputs.size(1)

		_, predicted = torch.max(out.data, 1) 
		train_accuracy.append((predicted == labels).sum().item() / predicted.size(0))

	epoch_loss_train = running_loss_train / train_batch
	
	print("Train accuracy: ",np.mean(train_accuracy))
	print(f'Train Loss: {epoch_loss_train:.8f}')

	if(epoch%10==0):
		torch.save(lmu.state_dict(), '/content/drive/Shared drives/CS726/PS_MNIST/bmu_h0_epoch'+str(epoch)+'.pt')
		torch.save(linear.state_dict(), '/content/drive/Shared drives/CS726/PS_MNIST/linear_bmu_h0_epoch'+str(epoch)+'.pt')

		



In [ ]:
# Testing Code	

def test_model(epoch):

	test_accuracy = []
	lmu.eval()
	linear.eval()
	batch_size = 1000
	running_loss_test = 0.0


	for batches in range(int(test_batch/batch_size)):

		inputs = images_test[batches*batch_size : (batches+1)*batch_size]
		labels = classes_test[batches*batch_size : (batches+1)*batch_size]
		
		hx = torch.zeros(batch_size, 256).to(device) # (batch, hidden_size)
		cx = torch.zeros(batch_size, 50).to(device)

		inputs = inputs.permute(1,0,2)

		for i in range(inputs.size()[0]):
			hx,cx = lmu(inputs[i],(hx,cx))

		output = linear(hx)  

		out = F.log_softmax(output, dim=1)

		loss =  F.cross_entropy(out,labels)

		running_loss_test += loss.item() * inputs.size(1)

		_, predicted = torch.max(out.data, 1) 

		test_accuracy.append((predicted == labels).sum().item() / predicted.size(0))


	epoch_loss_test = running_loss_test / test_batch
	
	print("Test accuracy: ",np.mean(test_accuracy))
	print(f'Test Loss: {epoch_loss_test:.8f}')


In [ ]:
num_epochs = 256

dataiter_train = iter(train_loader)
images_train, classes_train = dataiter_train.next()   
images_train = images_train.to(device)
classes_train = classes_train.to(device) 

dataiter_test = iter(test_loader)
images_test, classes_test = dataiter_test.next()   
images_test = images_test.to(device)
classes_test = classes_test.to(device) 
  
for epoch in range(num_epochs):
  print(f'Epoch {epoch}/{num_epochs}')
  print('-' * 10)
  train_model(epoch)
  test_model(epoch)

